In [ ]:
import matplotlib.pyplot as plt
import pytorch_lightning as L
import torch

from careamics.config import load_configuration
from careamics.dataset.prepare_dataset import (
    get_prediction_dataset,
    get_train_dataset,
    get_validation_dataset,
)
from careamics.lightning import LUNet
from careamics.prediction import stitch_prediction

In [ ]:
cfg = load_configuration(
    "/home/igor.zubarev/projects/caremics/examples/2D/n2v/n2v_2D_BSD.yml"
)
train_path = "/home/igor.zubarev/projects/caremics/examples/2D/data/denoising-N2V_BSD68.unzip/BSD68_reproducibility_data/train"
val_path = "/home/igor.zubarev/projects/caremics/examples/2D/data/denoising-N2V_BSD68.unzip/BSD68_reproducibility_data/val"
test_path = "/home/igor.zubarev/projects/caremics/examples/2D/data/denoising-N2V_BSD68.unzip/BSD68_reproducibility_data/test/image"

In [ ]:
train_dataset = get_train_dataset(cfg, train_path)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    num_workers=0,
    pin_memory=False,
)

val_dataset = get_validation_dataset(cfg, val_path)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=128,
    num_workers=0,
    pin_memory=False,
)

pred_dataset = get_prediction_dataset(
    cfg, pred_source=val_path, tile_shape=(128, 128), overlaps=(48, 48)
)
pred_dataloader = torch.utils.data.DataLoader(
    pred_dataset,
    batch_size=1,
    num_workers=0,
    pin_memory=False,
)

In [ ]:
model = LUNet(cfg)
trainer = L.Trainer(max_epochs=10)

In [ ]:
# trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
pred = trainer.predict(model, pred_dataloader)

In [ ]:
tiles = []
data = []
for p in pred:
    tiles.append(p[0])
    data.append(p[1][1:])

In [ ]:
out = stitch_prediction(tiles, data)

In [ ]:
plt.imshow(out)